In [1]:
import numpy as np
import pandas as pd
import cx_Oracle

In [2]:
rng = pd.period_range('2012-08-01', '2012-12-31', freq='M')
month_table = list(rng.astype(str))
print(month_table)

['2012-08', '2012-09', '2012-10', '2012-11', '2012-12']


In [3]:
def get_data_from_db(query):
    conn = cx_Oracle.connect('hoseo', 'hoseo', 'localhost:1521/xe')
    df = pd.read_sql(query, conn)
    conn.close()
    return df

In [28]:
for yearmon in month_table:
     print(f"""
            SELECT region, COUNT(*) AS "{yearmon}"
            FROM apt_deal_price
            WHERE TO_CHAR(contract_date, 'YYYY-MON') = '{yearmon}' 
            GROUP BY region 
            ORDER BY region ASC
            """)


            SELECT region, COUNT(*) AS "2012-08"
            FROM apt_deal_price
            WHERE TO_CHAR(contract_date, 'YYYY-MON') = '2012-08' 
            GROUP BY region 
            ORDER BY region ASC
            

            SELECT region, COUNT(*) AS "2012-09"
            FROM apt_deal_price
            WHERE TO_CHAR(contract_date, 'YYYY-MON') = '2012-09' 
            GROUP BY region 
            ORDER BY region ASC
            

            SELECT region, COUNT(*) AS "2012-10"
            FROM apt_deal_price
            WHERE TO_CHAR(contract_date, 'YYYY-MON') = '2012-10' 
            GROUP BY region 
            ORDER BY region ASC
            

            SELECT region, COUNT(*) AS "2012-11"
            FROM apt_deal_price
            WHERE TO_CHAR(contract_date, 'YYYY-MON') = '2012-11' 
            GROUP BY region 
            ORDER BY region ASC
            

            SELECT region, COUNT(*) AS "2012-12"
            FROM apt_deal_price
            WHERE TO_CHAR(cont

In [25]:
# 월별 지역별 전체 매매건수
def make_region_deal_count():
#    dataD = None
    for yearmon in month_table:

        query = f"""
            SELECT region, COUNT(*) AS "{yearmon}"
            FROM apt_deal_price
            WHERE TO_CHAR(contract_date, 'YYYY-MON') = '{yearmon}' 
            GROUP BY region 
            ORDER BY region ASC
            """

        queD = get_data_from_db(query)

        if yearmon == '2012-08':
            dataD = queD.copy()
        else:
            dataD = dataD.merge(queD)

#         if dataD == None:
#             dataD = queD.copy()
#         else:
#             dataD = dataD.merge(queD)


    dataD.info()
    dataD.to_csv('apt-data-cache-files/region_deal_count.csv', index=False, encoding='utf-8-sig')  

In [26]:
make_region_deal_count()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   REGION    0 non-null      object
 1   COUNT(*)  0 non-null      object
dtypes: object(2)
memory usage: 0.0+ bytes


In [27]:
testD = pd.read_csv('apt-data-cache-files/region_deal_count.csv')
testD

,REGION,COUNT(*)


In [8]:
# 국민임대주택 규모
# 전용면적 50m²(15평) 미만, 무주택 세대주 월평균 소득 50% 이하
# 전용면적 50m²(15평)이상∼60m²(18평) 이하, 무주택 세대주 월평균 소득 70% 이하

# 국민주택 규모
# 전용면적 85m²(25.7평) 이하
# 전용면적 100m²(30.2평) 이하, 수도권을 제외한 도시지역이 아닌 읍·면 지역

# 전용면적 100m²(30.2평)초과~135m²(40.8평)이하
# 전용면적 135m²(40.8평)초과

# ~ 50m²

# 한국감정원 기중 아파트 규모 분류

# ~ 60m² 초소형
# 60m² ~ 85m² 소형
# 85m² ~ 100m² 중소형
# 100m² ~ 135m² 중대형
# 135m² ~ 대형

In [9]:
# 지역 정보
region_table1 = [
    '강원도', '경기도', '경상남도', '경상북도', '광주광역시', '대구광역시',
    '대전광역시', '부산광역시', '서울특별시', '세종특별자치시', '울산광역시',
    '인천광역시', '전라남도', '전라북도', '제주특별자치도', '충청남도', '충청북도']

In [10]:
size_conditions = [
    ('under_50', 'adp.apt_size < 50'),
    ('50_60', 'adp.apt_size >= 50 AND adp.apt_size <= 60'),
    ('60_85', 'adp.apt_size > 60 AND adp.apt_size <= 85'),
    ('85_100', 'adp.apt_size > 85 AND adp.apt_size <= 100'),
    ('100_135', 'adp.apt_size > 100 AND adp.apt_size <= 135'),
    ('135_over', 'adp.apt_size > 135')]

col_apt_size_data = [
    'region', 'apt_size', 'y2012', 'y2013', 'y2014',
    'y2015', 'y2016', 'y2017', 'y2018', 'y2019', 'y2020']
    

In [11]:
# 지역별 사이즈별 매매건수 데이터 수집
def make_region_size_count(year = 2012):
    for cond in size_conditions:

        query = f"""
            SELECT adp.region AS region, COUNT(adp.apt_size) AS "{cond[0]}"
            FROM apt_deal_price adp
            WHERE to_char(adp.contract_date, 'YYYY') = '{year}' AND {cond[1]}
            GROUP BY adp.region
            ORDER BY adp.region ASC
            """

        queD = get_data_from_db(query)

        if cond[0] == 'under_50':
            dataD = queD.copy()
        else:
            dataD = dataD.merge(queD, how='left')
        
    dataD.fillna(0, inplace=True)
    dataD = dataD.astype( \
        {'under_50':'int64', '50_60':'int64', '60_85':'int64',
        '85_100':'int64', '100_135':'int64', '135_over':'int64'})
    
    # dataD.info()

    dataD.set_index('REGION', inplace=True)
    data_list = dataD.values.reshape(len(region_table1) * len(size_conditions), )

    return data_list


In [12]:
# 년도별 지역별 사이즈별 매매건수 데이터 만들기
def complete_region_size_count():

    # 최종 데이터 테이블 형태 만들기
    row_count = len(region_table1) * len(size_conditions)
    col_count = len(col_apt_size_data)
    allD = pd.DataFrame(np.zeros((row_count, col_count), dtype='int64'), columns=col_apt_size_data)

    size_list = [ x[0] for x in size_conditions ]
    allD['apt_size'] = size_list * len(region_table1)

    region_list = []
    for r in region_table1:
        region_list.extend([r] * len(size_conditions))
    allD['region'] = region_list

    # 년도별 데이터 수집
    for year in time_table2:
        data_list = make_region_size_count(year)
        allD['y' + year] = data_list
        
    allD.info()
    allD.to_csv('apt-sub-data-files/region_size_deal_count.csv', index=False, encoding='utf-8-sig')

In [13]:
# complete_region_size_count()

In [14]:
testD = pd.read_csv('apt-sub-data-files/region_size_deal_count.csv')
testD.head(12)

,region,apt_size,y2012,y2013,y2014,y2015,y2016,y2017,y2018,y2019,y2020
0,강원도,under_50,1745,5230,5375,5129,4973,4335,2953,2599,2222
1,강원도,50_60,2164,5709,6467,6971,6165,5092,4104,4500,4139
2,강원도,60_85,2063,5477,6316,7868,7326,5814,4712,5820,5144
3,강원도,85_100,75,194,218,278,262,198,135,154,160
4,강원도,100_135,432,894,992,1443,1282,937,742,660,704
5,강원도,135_over,151,326,239,349,343,232,179,181,181
6,경기도,under_50,5283,17442,23048,26504,23472,20073,17546,15751,16352
7,경기도,50_60,11330,40630,46031,52233,46651,40635,37919,35299,38755
8,경기도,60_85,17987,63772,73604,86951,75077,65850,67028,64718,69997
9,경기도,85_100,638,2393,3079,3755,3081,2675,2951,3153,3176


In [15]:
def get_yyyymm():
    query = """
        SELECT DISTINCT to_char(adp.contract_date, 'YYYYMM') AS ym
        FROM apt_deal_price adp
        ORDER BY ym ASC
        """

    queD = get_data_from_db(query)
    return list(queD['YM'])
    

In [16]:
# 월별 지역별 전체 매매건수
def make_region_deal_count_month():
    yyyymm = get_yyyymm()

    for ym in yyyymm:

        query = f"""
            SELECT adp.region, COUNT(*) AS ym{ym} 
            FROM apt_deal_price adp 
            WHERE TO_CHAR(adp.contract_date, 'YYYYMM') = '{ym}' 
            GROUP BY adp.region 
            ORDER BY adp.region ASC
            """

        queD = get_data_from_db(query)

        if ym == '201208':
            dataD = queD.copy()
        else:
            dataD = dataD.merge(queD, how='left')

    dataD.info()
    dataD.to_csv('apt-sub-data-files/region_deal_count_month.csv', index=False, encoding='utf-8-sig')

In [17]:
# make_region_deal_count_month()

In [18]:
testD = pd.read_csv('apt-sub-data-files/region_deal_count_month.csv')
testD.head()

,REGION,YM201208,YM201209,YM201210,YM201211,YM201212,YM201301,YM201302,YM201303,YM201304,...,YM201910,YM201911,YM201912,YM202001,YM202002,YM202003,YM202004,YM202005,YM202006,YM202007
0,강원도,933,1191,1495,1513,1498,1023,1227,2383,1660,...,1343,1377,1751,1439,1868,1579,1625,1982,2312,1745
1,경기도,5631,8083,10645,9178,8045,7115,8588,13003,14632,...,18101,20779,20988,20747,31851,16368,12759,16979,34952,17004
2,경상남도,1623,2113,3168,3101,2813,2286,2801,3747,4001,...,4026,4869,3822,3183,3704,2625,2633,3376,5764,3563
3,경상북도,1781,2087,2701,2519,2506,1947,2324,2837,3134,...,2549,2514,2400,2193,2689,1915,2083,3392,3843,2433
4,광주광역시,1204,1483,2012,1956,1732,1413,1799,2206,2051,...,2046,2133,2128,1954,2251,1738,1490,1778,2423,1831


## 아래 내용은 작업중..

In [19]:
# 월별 지역별 사이즈별 매매건수 데이터 수집
def make_region_size_count_month(ym = 201208):
    for cond in size_conditions:

        query = f"""
            SELECT adp.region AS region, COUNT(NVL(adp.apt_size, 0)) AS "{cond[0]}"
            FROM apt_deal_price adp
            WHERE to_char(adp.contract_date, 'YYYYMM') = '{ym}' AND {cond[1]}
            GROUP BY adp.region
            ORDER BY adp.region ASC
            """

        queD = get_data_from_db(query)

        if cond[0] == 'under_50':
            dataD = queD.copy()
        else:
            dataD = dataD.merge(queD, how='left')
    
    # dataD empty 처리

    dataD.fillna(0, inplace=True)
    dataD = dataD.astype( \
        {'under_50':'int64', '50_60':'int64', '60_85':'int64',
        '85_100':'int64', '100_135':'int64', '135_over':'int64'})
    
    # dataD.info()

    dataD.set_index('REGION', inplace=True)
    data_list = dataD.values.reshape(len(region_table1) * len(size_conditions), )

    return data_list

In [20]:
# 월별 지역별 사이즈별 매매건수 데이터 만들기
def complete_region_size_count():
    yyyymm = get_yyyymm()
    col_ym = [ 'ym' + y for y in yyyymm ]
    col_apt_size_month = ['region', 'apt_size']
    col_apt_size_month += col_ym

    # 최종 데이터 테이블 형태 만들기
    row_count = len(region_table1) * len(size_conditions)
    col_count = len(col_apt_size_month)
    allD = pd.DataFrame(np.zeros((row_count, col_count), dtype='int64'), columns=col_apt_size_month)

    size_list = [ x[0] for x in size_conditions ]
    allD['apt_size'] = size_list * len(region_table1)

    region_list = []
    for r in region_table1:
        region_list.extend([r] * len(size_conditions))
    allD['region'] = region_list

    # 년도별 데이터 수집
    for ym in yyyymm:
        data_list = make_region_size_count(ym)
        allD['ym' + ym] = data_list
        
    allD.info()
    allD.to_csv('sub-data-files/region_size_deal_count_month.csv', index=False, encoding='utf-8-sig')